## You will need the following 3 Packages

#### re, requests, json, openai, os, pandas


In [ ]:
#Uncomment and run the next line if you need to install or upgrade openai
# !pip install --upgrade openai

In [3]:
import re, requests, json, openai, os, pandas as pd
env = os.environ
OPENAI_PUBLIC_KEY = env['OPENAI_PUBLIC_KEY']
OPENAI_MSFT_KEY = env['OPENAI_MSFT_KEY']
openai.api_type = "azure"
openai.api_base = "https://dalle-exploration.openai.azure.com/"
openai.api_version = "2022-12-01"
openai.api_key = OPENAI_MSFT_KEY

# Add Environment variables.

### Here is how to set your environment variables. Start->edit system environment variable

<img align="left" src="AddEnvVariables.png" width=300 height=200 />

### Get your OPENAI API KEY from azure open ai console.

link = https://ms.portal.azure.com/#@microsoft.onmicrosoft.com/resource/subscriptions/67aa06b0-2686-40dc-92b0-1316ea0304d9/resourceGroups/Shared_AI_Experience/providers/Microsoft.CognitiveServices/accounts/DALLE-Exploration/cskeys
<img align="left" src="GetApiKeys.png" width=300 height=200 />


### Access Model via method 1 - Using openai library.
#### I have decided to always pass the name of the actual model and then have a look up that tells me the deployment name.
#### The deployment names as of 02/07 1 PM are in the image below but you can always access the lattest from this link
https://oai.azure.com/portal/e9d583b4c7a0488cb3e7acfbb439f609/deployment#

<img align="left" src="DeployedModels.png" width=300 height=200 />



In [4]:
#Define the way you want to get response from the model.

def get_promp_response_from_model(model, prompt, max_tokens = 4000, temperature = 1, top_p=0.5):
    
    lookup = {'text-davinci-002': 'text-davinci-002',
             'text-chat-davinci-002': 'chatGPTtest',
             'code-davinci-002': 'code-davinci-002',
             'text-davinci-003': 'test003',
             'text-ada-001': 'text-ada-001'}
    try:
        response  = openai.Completion.create(
          engine=lookup[model],
          prompt=prompt,
          temperature=temperature,
          max_tokens=max_tokens,
          top_p=top_p,
          frequency_penalty=0,
          presence_penalty=0,
          best_of=1,
          stop=None)
        return [model, prompt, response]
    except Exception as error:
        print(f'Exception ({error}) occurred')
        return  [model, prompt, error]
    
#Here is an example.
prompt = "How many times can the US fit in an African Map?"
response = get_promp_response_from_model('text-davinci-002', prompt)
print(response[2])

{
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n\nThe US can fit in an African Map approximately 3.5 times."
    }
  ],
  "created": 1675805401,
  "id": "cmpl-6hPpJKccQz4eyWqZk1SiLjAMtOwBi",
  "model": "text-davinci-002",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 12,
    "total_tokens": 28
  }
}


In [7]:
#You can access the response text by
responseText = response[2].choices[0].text 
print(responseText) #try ' '.join(responseText.strip().split())



The US can fit in an African Map approximately 3.5 times.


In [14]:
#Example using rest
OPENAI_PUBLIC_KEY = env['OPENAI_PUBLIC_KEY']
OPENAI_MSFT_KEY = env['OPENAI_MSFT_KEY']

base_url = "https://dalle-exploration.openai.azure.com"
deployment_name ="text-davinci-002"

def get_full_model_url(model):
    lookup = {'text-davinci-002': 'text-davinci-002',
             'text-chat-davinci-002': 'chatGPTtest',
             'code-davinci-002': 'code-davinci-002',
             'text-davinci-003': 'test003',
             'text-ada-001': 'text-ada-001'}
    assert model in lookup, f"Sorry, could not find a deployed model called '{model}'"
    deplyment_name = lookup[model]
    return f"{base_url}/openai/deployments/{deplyment_name}/completions?api-version=2022-12-01"

def form_headers_and_parameters(prompt,
          temperature=1,
          max_tokens=400,
          top_p=0.7,
          frequency_penalty=0,
          presence_penalty=0,
          best_of=1):
    data = {        
            "prompt":prompt, #Only required param
            'temperature':1,
            'max_tokens':400,
            'top_p':0.7,
            'frequency_penalty':0,
            'presence_penalty':0,
            'best_of':1    
    }

    headers={
            "api-key": OPENAI_MSFT_KEY,
            "Content-Type": "application/json"
        }
    return data, headers
    

In [15]:
data, headers = form_headers_and_parameters(prompt="What is the greatest invention ever?")

r = requests.post(get_full_model_url('text-davinci-002'), 
      headers=headers,
      json = data
    )

response = r.json()# or json.loads(r.text)
formatted_response = json.dumps(response, indent=4)
response

{'id': 'cmpl-6hQ7c5JIs3gEtnoiDDDE20SZJvWNt',
 'object': 'text_completion',
 'created': 1675806536,
 'model': 'text-davinci-002',
 'choices': [{'text': '\n\nThe greatest invention ever is the wheel.',
   'index': 0,
   'finish_reason': 'stop',
   'logprobs': None}],
 'usage': {'completion_tokens': 10, 'prompt_tokens': 7, 'total_tokens': 17}}

In [ ]:
#Will be adding more apis as time goes on.